In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
import ta
from fucnt_opt import TradingStrategyOptimizer_DNN
from fucnt_opt import clean_ds
from fucnt_opt import dnn_strategy
from CNN import TradingStrategyOptimizer_CNN
from CNN import cnn_strategy
from RNN import TradingStrategyOptimizer_RNN
from RNN import rnn_strategy

In [ ]:
optimizer_dnn = TradingStrategyOptimizer_DNN('../data/close_data_buy_5.csv', '../data/close_data_sell_5.csv')
df_dnn, model_dnn_buy, model_dnn_sell = optimizer_dnn.run_dnn()
optimizer_cnn = TradingStrategyOptimizer_CNN('../data/close_data_buy_5.csv', '../data/close_data_sell_5.csv')
df_cnn, model_cnn_buy, model_cnn_sell = optimizer_cnn.run_cnn()
optimizer_rnn = TradingStrategyOptimizer_RNN('../data/close_data_buy_5.csv', '../data/close_data_sell_5.csv')
df_rnn, model_rnn_buy, model_rnn_sell = optimizer_rnn.run_rnn()

In [ ]:


df1 = pd.read_csv("../data/aapl_5m_train.csv")
df_5min = clean_ds(df1)
close_data = df_5min[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']]
close_data = close_data.dropna()

df_indexed_dnn = df_dnn.reset_index()
df_indexed_cnn = df_cnn.reset_index()
df_indexed_rnn = df_rnn.reset_index()

close_data_indexed_dnn = close_data.reset_index()

close_data_updated = close_data_indexed_dnn.join(df_indexed_dnn[['Y_BUY_PRED_DNN', 'Y_SELL_PRED_DNN']])
close_data_updated = close_data_updated.join(df_indexed_cnn[['Y_BUY_PRED_CNN', 'Y_SELL_PRED_CNN']], rsuffix='_CNN')
close_data_updated = close_data_updated.join(df_indexed_rnn[['Y_BUY_PRED_RNN', 'Y_SELL_PRED_RNN']], rsuffix='_RNN')


closes_5min = close_data_updated[['Timestamp', 'Close','Y_BUY_PRED_DNN', 'Y_SELL_PRED_DNN', 'Y_BUY_PRED_CNN', 'Y_SELL_PRED_CNN', 'Y_BUY_PRED_RNN', 'Y_SELL_PRED_RNN']]
closes_5min


In [ ]:



cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss = trial.suggest_float('stop_loss', 0.00250, 0.05)
    take_profit = trial.suggest_float('take_profit', 0.00250, 0.05)
    n_shares = trial.suggest_int('n_shares', 5, 200)
    
    dnn_strat = dnn_strategy(
        df=closes_5min,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss=stop_loss,
        take_profit=take_profit
    )
    cnn_strat = cnn_strategy(
        df=closes_5min,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss=stop_loss,
        take_profit=take_profit
    )
    rnn_strat = rnn_strategy(
        df=closes_5min,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss=stop_loss,
        take_profit=take_profit
    )

    
    strategy_values = {
        'dnn_strategy': dnn_strat.run_strategy_dnn(),
        'cnn_strategy': cnn_strat.run_strategy_cnn(),
        'rnn_strategy': rnn_strat.run_strategy_rnn()
    }
    
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [ ]:
# Inicializar y ejecutar la optimización

study = optuna.create_study(direction='maximize')
study.optimize(optimize ,n_trials=10,n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

In [ ]:
from sklearn.preprocessing import StandardScaler

df_tets = clean_ds(pd.read_csv('../data/aapl_5m_test.csv'))
model_buy = load_model('model_buy_rnn.keras')
model_sell = load_model('model_sell_rnn.keras')

close_data_test = df_tets[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']].dropna()
cl_dat_test = close_data_test.copy()

x_buy_test = cl_dat_test.drop(columns=['Timestamp','Y_BUY']).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_buy_test)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

close_data_test['Y_BUY_PRED_RNN'] = model_rnn_buy.predict(x_buy_test)
close_data_test = close_data_test.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3','X_t-4' ,'X_t-5'])
close_data_test['Y_SELL_PRED_RNN'] = model_rnn_sell.predict(X_reshaped)
